Pruebas sobre el dataset 

In [1]:
import pandas as pd

# Cargar Corpus y formatear fechas

In [4]:
file_path = "Corpus_completo_revisado.xlsx"
corpus_completo = pd.read_excel(file_path)

corpus_completo.head()

,Diario,Autor,Fecha,Título,Texto,Vínculo,ID
0,El Espectador,Gonzalo Hernández,1 de enero de 2018,Fajardo: para nada tibio,"La Coalición Colombia Partido Alianza Verde, ...",https://web.archive.org/web/20180102104221/htt...,1
1,El Espectador,Eduardo Barajas Sandoval,1 de enero de 2018,Macedonia de Norte,Las interpretaciones de la historia sirven com...,https://web.archive.org/web/20180102104221/htt...,2
2,El Espectador,Daniel Emilio Rojas Castro,1 de enero de 2018,El nacionalismo según Vargas Llosa,La semana pasada Mario Vargas Llosa publicó un...,https://web.archive.org/web/20180102104221/htt...,3
3,El Espectador,Reinaldo Spitaletta,1 de enero de 2018,"Tiempo sagrado, tiempo profano","Pudiera decirse, sin ser una verdad absoluta, ...",https://web.archive.org/web/20180102104221/htt...,4
4,El Espectador,Aura Lucía Mera,1 de enero de 2018,La rebelión de los bueyes,Lo mejor del encierro de Las Ventas fueron los...,https://web.archive.org/web/20180102104221/htt...,5


# Búsqueda exacta de  tesauros

# Búsqueda semántica de tesauros